Contagem da quantidade de crimes violentos ocoridos em Minas Gerais em 2023 (por natureza do crime)

In [2]:
import pandas as pd

df = pd.read_csv('../data/bruto/crimes_violentos_2024.csv', sep= ';')

df.drop_duplicates()

contCrimes = df['natureza'].value_counts()

print(contCrimes)


natureza
Estupro Consumado                        10236
Estupro de Vulnerável Consumado          10236
Estupro de Vulnerável Tentado            10236
Estupro Tentado                          10236
Extorsão Consumado                       10236
Extorsão Mediante Sequestro Consumado    10236
Extorsão Tentado                         10236
Homicídio Tentado                        10236
Roubo Consumado                          10236
Roubo Tentado                            10236
Sequestro e Cárcere Privado Consumado    10236
Sequestro e Cárcere Privado Tentado      10236
Homicídio Consumado (Registros)          10236
Name: count, dtype: int64


Isso acontece porque o arquivo está estruturado com blocos idênticos:
para cada tipo de crime, há uma linha para cada município de Minas Gerais, sempre com o mesmo número de registros (10236).

Crimes por Regiões Integradas de Segurança Pública (RISP)

In [ ]:
import pandas as pd

df = pd.read_csv('../data/bruto/crimes_violentos_2024.csv', sep= ';')

df.drop_duplicates()

crimes_regiao = df.groupby('risp')['natureza'].count().sort_values(ascending=False)

crimes_regiao = crimes_regiao.reset_index()
crimes_regiao.columns = ['RISP', 'TOTAL_CRIMES']

print(crimes_regiao)

crimes_regiao.to_csv('../data/processado/crimes_por_regiao.csv', sep=';', index=False, encoding='utf-8')




    RISP  TOTAL_CRIMES
0     12         15132
1      4         13416
2     11         12012
3     17         11232
4     13          9516
5     15          9360
6      8          9048
7     18          8580
8     14          8112
9      7          7800
10     6          6864
11     5          4680
12    10          3588
13     2          2808
14     3          2808
15     9          2808
16    16          2652
17    19          2496
18     1           156


Unir Tabela de Crimes com tabela de códigos de regiões

Inserir as coordenadas da cede de cada região (RISP) segundo o Google Maps

In [59]:
import pandas as pd

df = pd.read_csv('../data/processado/crimes_por_regiao.csv', sep=';')

coords = {
    1:  [-19.9167, -43.9345],  # Belo Horizonte
    2:  [-19.7668, -44.0868],  # Ribeirão das Neves
    3:  [-19.6944, -43.9237],  # Vespasiano
    4:  [-21.7642, -43.3496],  # Juiz de Fora
    5:  [-19.7479, -47.9319],  # Uberaba
    6:  [-21.2475, -44.9997],  # Lavras
    7:  [-20.1452, -44.8912],  # Divinópolis
    8:  [-18.8545, -41.9459],  # Governador Valadares
    9:  [-18.9128, -48.2755],  # Uberlândia
    10: [-19.0003, -46.3162],  # Carmo do Paranaíba 
    11: [-16.7286, -43.8578],  # Montes Claros
    12: [-19.4820, -42.5279],  # Ipatinga
    13: [-21.2250, -43.7737],  # Barbacena
    14: [-18.7573, -44.4303],  # Curvelo
    15: [-17.8576, -41.5059],  # Teófilo Otoni
    16: [-16.3595, -46.9067],  # Unaí
    17: [-22.2300, -45.9380],  # Pouso Alegre
    18: [-21.7890, -46.5663],  # Poços de Caldas
    19: [-19.4671, -44.2469],  # Sete Lagoas
}

df['latitude'] = df['RISP'].map(lambda x: coords.get(x, [None, None])[0])
df['longitude'] = df['RISP'].map(lambda x: coords.get(x, [None, None])[1])

df.to_csv('../data/processado/crimes_por_regiao.csv', sep=';', index=False, encoding='utf-8')

#criação do database final
df.to_csv('../data/processado/database_rispMG.csv', sep=';', index=False, encoding='utf-8')


Adição da população de cada RISP no dataBase

In [ ]:
import pandas as pd

#novos dados a serem adicionados 
dadosPop = pd.read_csv('../data/bruto/popTotal_2022.csv', sep=';', encoding= 'utf-8')

#relação municipio - RISP
risp = pd.read_csv('../data/bruto/municipios_risp.csv', sep=';', encoding= 'utf-8')

#arquivo onde serão adicionados os dados 
df = pd.read_csv('../data/processado/database_rispMG.csv', sep=';', encoding= 'utf-8')

risp.columns = risp.columns.str.strip().str.lower()
dadosPop.columns = dadosPop.columns.str.strip().str.lower()
df.columns = df.columns.str.strip().str.lower()

relacao = risp.merge(dadosPop, on='município', how='inner')

popRisp = relacao.groupby("risp")["2022"].sum().reset_index()


dfPop = df.merge(popRisp, on="risp", how='left')


print(dfPop)

dfPop.to_csv('../data/processado/database_rispMG.csv', sep=';', encoding= 'utf-8', index=False)





    risp  total_crimes  latitude  longitude     2022
0     12         15132  -19.4820   -42.5279  1718360
1      4         13416  -21.7642   -43.3496  1574197
2     11         12012  -16.7286   -43.8578  1427455
3     17         11232  -22.2300   -45.9380  1023552
4     13          9516  -21.2250   -43.7737   732771
5     15          9360  -17.8576   -41.5059   825657
6      8          9048  -18.8545   -41.9459   687921
7     18          8580  -21.7890   -46.5663  1105320
8     14          8112  -18.7573   -44.4303   673977
9      7          7800  -20.1452   -44.8912  1182836
10     6          6864  -21.2475   -44.9997   880430
11     5          4680  -19.7479   -47.9319   823093
12    10          3588  -19.0003   -46.3162   534508
13     2          2808  -19.7668   -44.0868  1905395
14     3          2808  -19.6944   -43.9237   988106
15     9          2808  -18.9128   -48.2755  1100550
16    16          2652  -16.3595   -46.9067   377579
17    19          2496  -19.4671   -44.2469   

Ordenando os Dados por Código Risp

In [68]:
import pandas as pd

df = pd.read_csv('../data/processado/database_rispMG.csv', sep=';', encoding= 'utf-8')
df = df.sort_values(by="risp", ascending=True)

df.to_csv('../data/processado/database_rispMG.csv', sep=';', encoding= 'utf-8', index=False)

print(df)

    risp  total_crimes  latitude  longitude  populacao
18     1           156  -19.9167   -43.9345    2315560
13     2          2808  -19.7668   -44.0868    1905395
14     3          2808  -19.6944   -43.9237     988106
1      4         13416  -21.7642   -43.3496    1574197
11     5          4680  -19.7479   -47.9319     823093
10     6          6864  -21.2475   -44.9997     880430
9      7          7800  -20.1452   -44.8912    1182836
6      8          9048  -18.8545   -41.9459     687921
15     9          2808  -18.9128   -48.2755    1100550
12    10          3588  -19.0003   -46.3162     534508
2     11         12012  -16.7286   -43.8578    1427455
0     12         15132  -19.4820   -42.5279    1718360
4     13          9516  -21.2250   -43.7737     732771
8     14          8112  -18.7573   -44.4303     673977
5     15          9360  -17.8576   -41.5059     825657
16    16          2652  -16.3595   -46.9067     377579
3     17         11232  -22.2300   -45.9380    1023552
7     18  

Adicionando dados de renda per capita

In [89]:
import pandas as pd

rendaMunicipio = pd.read_csv('../data/bruto/renda_per_capita_2010.csv', sep=';', encoding= 'utf-8')

risp = pd.read_csv('../data/bruto/municipios_risp.csv', sep=';', encoding= 'utf-8')

df = pd.read_csv('../data/processado/database_rispMG.csv', sep=';', encoding= 'utf-8')

risp.columns = risp.columns.str.strip().str.lower()
rendaMunicipio.columns = rendaMunicipio.columns.str.strip().str.lower()
df.columns = df.columns.str.strip().str.lower()

relacao = risp.merge(rendaMunicipio, on='município', how='inner')
relacao['2010'] = relacao['2010'].astype(str).str.replace(',', '.')
relacao['2010'] = pd.to_numeric(relacao['2010'], errors='coerce')

rendaRisp = relacao.groupby("risp")["2010"].mean().reset_index()
#Fator de correção segundo: https://www.idinheiro.com.br/
fatorIPCA = 2.2315

rendaRisp['2010'] = rendaRisp['2010'] * fatorIPCA

dfRenda = df.merge(rendaRisp, on="risp", how='left')

print(dfRenda)

dfRenda.to_csv('../data/processado/database_rispMG.csv', sep=';', encoding= 'utf-8', index=False)


    risp  total_crimes  latitude  longitude  populacao         2010
0      1           156  -19.9167   -43.9345    2315560  3341.202635
1      2          2808  -19.7668   -44.0868    1905395  1305.204350
2      3          2808  -19.6944   -43.9237     988106  1495.459415
3      4         13416  -21.7642   -43.3496    1574197  1150.614633
4      5          4680  -19.7479   -47.9319     823093  1603.196628
5      6          6864  -21.2475   -44.9997     880430  1268.971890
6      7          7800  -20.1452   -44.8912    1182836  1361.878648
7      8          9048  -18.8545   -41.9459     687921   843.576091
8      9          2808  -18.9128   -48.2755    1100550  1500.402333
9     10          3588  -19.0003   -46.3162     534508  1487.994277
10    11         12012  -16.7286   -43.8578    1427455   663.031120
11    12         15132  -19.4820   -42.5279    1718360   981.789731
12    13          9516  -21.2250   -43.7737     732771  1031.525481
13    14          8112  -18.7573   -44.4303     

Adição do IDH Médio de cada RISP

In [94]:
import pandas as pd

dadosIDH = pd.read_csv('../data/bruto/IDHM_2010.csv', sep=';', encoding= 'utf-8')

risp = pd.read_csv('../data/bruto/municipios_risp.csv', sep=';', encoding= 'utf-8')

df = pd.read_csv('../data/processado/database_rispMG.csv', sep=';', encoding='utf-8')

risp.columns = risp.columns.str.strip().str.lower()
dadosIDH.columns = dadosIDH.columns.str.strip().str.lower()
df.columns = df.columns.str.strip().str.lower()

relacao = risp.merge(dadosIDH, on='município', how='inner')
relacao['2010'] = relacao['2010'].astype(str).str.replace(',', '.')
relacao['2010'] = pd.to_numeric(relacao['2010'], errors='coerce')

IDHRisp = relacao.groupby("risp")["2010"].mean().reset_index().round(3)

dfIDH = df.merge(IDHRisp, on="risp", how='left')

print(dfIDH)

dfIDH.to_csv('../data/processado/database_rispMG.csv', sep=';', encoding= 'utf-8', index=False)




    risp  total_crimes  latitude  longitude  populacao  renda_media   2010
0      1           156  -19.9167   -43.9345    2315560  3341.202635  0.810
1      2          2808  -19.7668   -44.0868    1905395  1305.204350  0.695
2      3          2808  -19.6944   -43.9237     988106  1495.459415  0.715
3      4         13416  -21.7642   -43.3496    1574197  1150.614633  0.673
4      5          4680  -19.7479   -47.9319     823093  1603.196629  0.712
5      6          6864  -21.2475   -44.9997     880430  1268.971890  0.695
6      7          7800  -20.1452   -44.8912    1182836  1361.878648  0.702
7      8          9048  -18.8545   -41.9459     687921   843.576091  0.630
8      9          2808  -18.9128   -48.2755    1100550  1500.402333  0.714
9     10          3588  -19.0003   -46.3162     534508  1487.994277  0.707
10    11         12012  -16.7286   -43.8578    1427455   663.031120  0.623
11    12         15132  -19.4820   -42.5279    1718360   981.789731  0.654
12    13          9516  -

Adição de informações de educação

In [ ]:
import pandas as pd

dadosAnalfabetismo = risp = pd.read_csv('../data/bruto/analfabetismo_maior_15.csv', sep=';', encoding= 'utf-8')

risp = pd.read_csv('../data/bruto/municipios_risp.csv', sep=';', encoding= 'utf-8')

df = pd.read_csv('../data/processado/database_rispMG.csv', sep=';', encoding='utf-8')

pop = pd.read_csv('../data/bruto/popTotal_2022.csv', sep=';', encoding='utf-8')

risp.columns = risp.columns.str.strip().str.lower()
dadosAnalfabetismo.columns = dadosAnalfabetismo.columns.str.strip().str.lower()
df.columns = df.columns.str.strip().str.lower()
pop.columns = pop.columns.str.strip().str.lower()

relacao = risp.merge(dadosAnalfabetismo, on='município', how='inner')
relacao['taxa'] = relacao['taxa'].astype(str).str.replace(',', '.')
relacao['taxa'] = pd.to_numeric(relacao['taxa'], errors='coerce')

relacao = relacao.merge(pop, on='município', how='inner')
relacao['2022'] = relacao['2022'].astype(str).str.replace(',', '.')
relacao['2022'] = pd.to_numeric(relacao['2022'], errors='coerce')

taxaRisp = (
    relacao.groupby('risp')
    .apply(lambda x: (x['taxa'] * x['2022']).sum() / x['2022'].sum())
    .reset_index(name='taxa_de_analfabetismo(%)')
).round(3)

dfAnalf = df.merge(taxaRisp, on="risp", how='left')

print(dfAnalf)

dfAnalf.to_csv('../data/processado/database_rispMG.csv', sep=';', encoding= 'utf-8', index=False)



    risp  total_crimes  latitude  longitude  populacao  renda_media  \
0      1           156  -19.9167   -43.9345    2315560  3341.202635   
1      2          2808  -19.7668   -44.0868    1905395  1305.204350   
2      3          2808  -19.6944   -43.9237     988106  1495.459415   
3      4         13416  -21.7642   -43.3496    1574197  1150.614633   
4      5          4680  -19.7479   -47.9319     823093  1603.196629   
5      6          6864  -21.2475   -44.9997     880430  1268.971890   
6      7          7800  -20.1452   -44.8912    1182836  1361.878648   
7      8          9048  -18.8545   -41.9459     687921   843.576091   
8      9          2808  -18.9128   -48.2755    1100550  1500.402333   
9     10          3588  -19.0003   -46.3162     534508  1487.994277   
10    11         12012  -16.7286   -43.8578    1427455   663.031120   
11    12         15132  -19.4820   -42.5279    1718360   981.789731   
12    13          9516  -21.2250   -43.7737     732771  1031.525481   
13    

C:\Users\Meu computador\AppData\Local\Temp\ipykernel_4196\1918415745.py:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: (x['taxa'] * x['2022']).sum() / x['2022'].sum())


Adição da Expectativa de vida

In [6]:
import pandas as pd

risp = pd.read_csv('../data/bruto/municipios_risp.csv', sep=';', encoding= 'utf-8')

df = pd.read_csv('../data/processado/database_rispMG.csv', sep=';', encoding='utf-8')

expcVida = pd.read_csv('../data/bruto/expecVida_2010.csv', sep=';', encoding='utf-8')

risp.columns = risp.columns.str.strip().str.lower()
expcVida.columns = expcVida.columns.str.strip().str.lower()
df.columns = df.columns.str.strip().str.lower()


relacao = risp.merge(expcVida, on='município', how='inner')
relacao['esperança_vida'] = relacao['esperança_vida'].astype(str).str.replace(',', '.')
relacao['esperança_vida'] = pd.to_numeric(relacao['esperança_vida'], errors='coerce')

expecVidaRisp = relacao.groupby("risp")["esperança_vida"].mean().reset_index().round(3)

dfExpecVida = df.merge(expecVidaRisp, on="risp", how='left')

print(dfExpecVida)

dfExpecVida.to_csv('../data/processado/database_rispMG.csv', sep=';', encoding= 'utf-8', index=False)

    risp  total_crimes  latitude  longitude  populacao  renda_media  \
0      1           156  -19.9167   -43.9345    2315560     3341.200   
1      2          2808  -19.7668   -44.0868    1905395     1305.202   
2      3          2808  -19.6944   -43.9237     988106     1495.460   
3      4         13416  -21.7642   -43.3496    1574197     1150.620   
4      5          4680  -19.7479   -47.9319     823093     1603.200   
5      6          6864  -21.2475   -44.9997     880430     1268.970   
6      7          7800  -20.1452   -44.8912    1182836     1361.880   
7      8          9048  -18.8545   -41.9459     687921      843.580   
8      9          2808  -18.9128   -48.2755    1100550     1500.400   
9     10          3588  -19.0003   -46.3162     534508     1487.990   
10    11         12012  -16.7286   -43.8578    1427455      663.030   
11    12         15132  -19.4820   -42.5279    1718360      981.790   
12    13          9516  -21.2250   -43.7737     732771     1031.530   
13    

Arrumando o tipo das colunas

In [3]:
import pandas as pd

df = pd.read_csv('../data/processado/database_rispMG.csv', sep=';', encoding='utf-8')

df['total_crimes'] = pd.to_numeric(df['total_crimes'], errors='coerce')
df['populacao'] = pd.to_numeric(df['populacao'], errors='coerce')

df.to_csv('../data/processado/database_rispMG.csv', sep=';', encoding= 'utf-8', index=False)